In [326]:
# import packages
import json
import os
import praw
import requests
import datetime as dt
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm import tqdm

In [5]:
# API arguments
CLIENT_ID = '8InJgsdPcLSGUQ'
CLIENT_SECRET = 'MFjtDa8dUZf-Cq75UHl2XMfQmoA'
USER_AGENT = 'subreddit top post scraping by /u/samjabroni'
USERNAME = 'samjabroni'
PASSWORD = 'cxFw$u#x55AdkBebmG7D'

In [6]:
# create API object
reddit = praw.Reddit(client_id=CLIENT_ID,
                     client_secret=CLIENT_SECRET,
                     user_agent=USER_AGENT,
                     username=USERNAME,
                     password=PASSWORD)

Version 7.0.0 of praw is outdated. Version 7.1.0 was released Tuesday June 23, 2020.


In [128]:
# Reference: https://www.storybench.org/how-to-scrape-reddit-with-python/

#def make_subreddit_df(subreddit, limit):
    
#    subreddit = reddit.subreddit(subreddit)

#    top_posts = subreddit.top(limit=limit)

#    post_dict = { "title":[], "score":[],
#                 "id":[], "url":[], "num_comments":[],
#                 "created":[]}

#    for submission in top_posts:
#        post_dict["title"].append(submission.title)
#        post_dict["score"].append(submission.score)
#        post_dict["id"].append(submission.id)
#        post_dict["url"].append(submission.url)
#        post_dict["num_comments"].append(submission.num_comments)
#        post_dict["created"].append(dt.datetime.fromtimestamp(submission.created))
    
#    subreddit_df = pd.DataFrame(data=post_dict)
    
#    return subreddit_df

In [146]:
# get dataframes
fp_df = make_subreddit_df("FoodPorn", limit=1000)
sfp_df = make_subreddit_df("shittyfoodporn", limit=1000)

In [ ]:
#fp_df.to_csv('Data/foodporn.csv', index=False)
#fp_df.to_csv('Data/shittyfoodporn.csv', index=False)

In [184]:
os.getcwd()

'/home/sam/Desktop/FoodPorn'

In [183]:
os.chdir('..')

In [ ]:
# stream submissions
for submission in reddit.subreddit("all").stream.submissions():
    print(submission)

In [287]:
def get_top_df(subreddit, n_pages=1):
    
    headers = {'User-Agent': 'Mozilla/5.0'}
    n = 0
    url = 'https://old.reddit.com/r/' + str(subreddit) +'/top.json'
    
    post_dict = { "title":[], "created":[],
                 "id":[], "url":[],
                 "score":[], "num_comments":[]}
    
    while n < n_pages:
        try:
            r = requests.get(url, headers=headers)
            js = r.json()
        
            for post in range(len(js['data']['children'])):
                post_dict['title'].append(js['data']['children'][post]['data']['title'])
                post_dict['score'].append(js['data']['children'][post]['data']['score'])
                post_dict['id'].append(js['data']['children'][post]['data']['id'])
                post_dict['url'].append(js['data']['children'][post]['data']['url'])
                post_dict['num_comments'].append(js['data']['children'][post]['data']['num_comments'])
                post_dict['created'].append(dt.datetime.fromtimestamp(js['data']['children'][post]['data']['created']))
            
            url = 'https://old.reddit.com/r/' + str(subreddit) + '.json' + '?count=' + str(25*(n+1)) + '&after=' + str(js['data']['after']) 
        
        except:
            print(f'error at page {n}')
            continue
        
        n += 1
        time.sleep(3)
        
    subreddit_df = pd.DataFrame(data=post_dict)
    
    return subreddit_df

In [288]:
fp_df = get_top_df('FoodPorn',800)

In [299]:
#test = []
#for url in fp_df.url:
#    test.append(str.split(str(url),'.')[-1])
#testdf = pd.DataFrame(test)
#testdf.value_counts()

jpg                   18818
png                     315
gif                      44
jpeg                     23
com/vL3kddD              23
com/tVLIORa              23
com/H9ai8dW              23
com/Ti2VhnS              23
com/VDE3Tr0              22
com/3Tvd9Yn              22
com/6HEr5ZZ              22
com/COdUFsE              22
com/IKJuKQz              22
com/dMsqkIP              22
com/ZzjXfEq              22
com/gallery/iwpvob       22
com/hku43QJ              22
com/kmCpEqO              22
com/ooxJzeV              22
com/uCXOz0T              22
com/2SKje5q              22
com/YpKZCAA               9
dtype: int64

In [293]:
fp_df.to_csv('Data/foodporn.csv', index=False)

In [289]:
fp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19587 entries, 0 to 19586
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   title         19587 non-null  object        
 1   created       19587 non-null  datetime64[ns]
 2   id            19587 non-null  object        
 3   url           19587 non-null  object        
 4   score         19587 non-null  int64         
 5   num_comments  19587 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 918.3+ KB


In [294]:
sfp_df = get_top_df('shittyfoodporn',800)

In [295]:
sfp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19754 entries, 0 to 19753
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   title         19754 non-null  object        
 1   created       19754 non-null  datetime64[ns]
 2   id            19754 non-null  object        
 3   url           19754 non-null  object        
 4   score         19754 non-null  int64         
 5   num_comments  19754 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 926.1+ KB


In [296]:
sfp_df.to_csv('Data/shittyfoodporn.csv', index=False)

In [343]:
os.chdir('FoodPorn')
os.getcwd()

'/home/sam/Desktop/FoodPorn'

In [345]:
def download_subreddit_images(subreddit_df, directory):
    """downloads all static images (png & jpg) from specified subreddit dataframe to desired directory"""
    
    headers = {'user-agent': 'image_downloader'}
    url_dict = {}
    
    # checks if directory exists, if not makes one
    current_path = os.getcwd()
    if not os.path.exists(directory):
        os.makedirs(directory)
    os.chdir(directory)
    
    # find urls and create file names from unique post id
    for row in subreddit_df.itertuples():
        filename = f"{row[0]}.{str.split(str(row[4]), sep='.')[-1]}"
        url =row[4]
        url_dict[filename] = url
    
    # iterates over static images in dict and saves to file
    for filename, url in tqdm(url_dict.items()):
        # check if url is png or jpg
        if (str.split(filename, sep='.')[-1] == 'jpg') or (str.split(filename, sep='.')[-1] =='png'):
            r = requests.get(url, headers=headers, stream=True)
       
            with open(filename, 'wb') as fd:
                for chunk in r.iter_content(chunk_size=128):
                    fd.write(chunk)
            time.sleep(3)
        pass
    
    os.chdir(current_path)

In [332]:
download_subreddit_images(fp_df,'Images/Food_Porn')

100%|██████████| 19587/19587 [6:48:57<00:00,  1.25s/it]  


In [349]:
download_subreddit_images(sfp_df.iloc[4331:],'Images/Shitty_Food_Porn')

100%|██████████| 15423/15423 [13:27:09<00:00,  3.14s/it]  


EDA ideas
Pixel wise absolute difference
Color channels
Contrast
Intensity
Color change within image
Text?

size - by pixels (and resolution?)